# Data Cleaning

In [34]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import os
import sqlite3
from IPython.display import display

# load custom modules
from data_sampler import sampler
from sqlite_to_csv import sqlite_to_csv
from reading_tools import read_data_from_db

## 1. Clean the *weather* Data 

### Display the *weather* Data

In [28]:
# read the data
df_weather = pd.read_csv("../data/weather.csv", parse_dates=["date"])

# display the data
display(df_weather.head())

# show the number of data points in trip.csv
print("This table has {npnts} data points and {col} columns.".format(npnts=df_weather.shape[0], col=df_weather.shape[1]))

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,2013-08-29,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,2013-08-30,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,2013-08-31,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,2013-09-01,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,2013-09-02,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


This table has 3665 data points and 24 columns.


### Fix _weather_ Data

#### Fix Data Type Mismatch in *_precipitation_inches* Column

In [5]:
# show the data types of the columns
df_weather.dtypes

date                              datetime64[ns]
max_temperature_f                        float64
mean_temperature_f                       float64
min_temperature_f                        float64
max_dew_point_f                          float64
mean_dew_point_f                         float64
min_dew_point_f                          float64
max_humidity                             float64
mean_humidity                            float64
min_humidity                             float64
max_sea_level_pressure_inches            float64
mean_sea_level_pressure_inches           float64
min_sea_level_pressure_inches            float64
max_visibility_miles                     float64
mean_visibility_miles                    float64
min_visibility_miles                     float64
max_wind_Speed_mph                       float64
mean_wind_speed_mph                      float64
max_gust_speed_mph                       float64
precipitation_inches                      object
cloud_cover         

From the data types of the columns displayed above we can see that _pandas_ considered *precipitation_inches* as an object instead of a float. Lets examine the *precipitation_inches* column.  

In [9]:
# display unique values in precipitation_inches column
df_weather.precipitation_inches.unique()

array(['0', '0.23', 'T', '0.01', '0.28', '0.63', '0.29', '0.06', '0.85',
       '0.09', '0.64', '0.42', '0.35', '0.43', '0.22', '0.74', '0.03',
       '0.12', '0.16', '0.49', '0.17', '0.08', '0.04', '0.53', '0.07',
       '0.02', '0.83', '1.06', '1.71', '0.37', '0.27', '0.45', '0.78',
       '0.88', '0.66', '0.47', '0.1', '0.61', '0.14', '0.05', '0.68',
       '0.97', '0.26', '0.15', '0.87', '0.57', '0.69', '0.32', '0.21',
       '0.24', '0.52', '0.36', '0.33', '0.25', '0.11', '0.2', '1.18',
       '1.43', '3.12', '0.48', '0.19', '1.09', '0.65', '0.13', '0.91',
       '0.99', '0.18', '0.4', '1.07', nan, '0.41', '0.34', '1.25', '1.85',
       '3.36', '0.71', '1.3', '0.72', '0.6', '0.51', '1.2', '1.28', '3.23',
       '0.55', '1.26', '0.39'], dtype=object)

The unique values in *precipitation_inches* shows that there is "T" among other floating point numbers. Documentation about the *weather* data reads that " 'T'= trace when amount less than .01 inch". That means "T" represent any number between 0 and 0.01 inch. Since less than 0.01 inch of precipitation is considered to be very light, we may replace "T" with any number less that 0.01. Below we replace it with 0.005 inch.

In [23]:
# change "T" to "0.005" in precipitation_inches.
# "T"= trace when amount less than .01 inch"
df_weather.loc[:, "precipitation_inches"] = df_weather.precipitation_inches.apply(lambda x: "0.005" if x == "T" else x)

# display unique values in precipitation_inches column after changing "T" into "0.005"
np.sort(df_weather.precipitation_inches.unique())

array([nan, '0', '0.005', '0.01', '0.02', '0.03', '0.04', '0.05', '0.06',
       '0.07', '0.08', '0.09', '0.1', '0.11', '0.12', '0.13', '0.14',
       '0.15', '0.16', '0.17', '0.18', '0.19', '0.2', '0.21', '0.22',
       '0.23', '0.24', '0.25', '0.26', '0.27', '0.28', '0.29', '0.32',
       '0.33', '0.34', '0.35', '0.36', '0.37', '0.39', '0.4', '0.41',
       '0.42', '0.43', '0.45', '0.47', '0.48', '0.49', '0.51', '0.52',
       '0.53', '0.55', '0.57', '0.6', '0.61', '0.63', '0.64', '0.65',
       '0.66', '0.68', '0.69', '0.71', '0.72', '0.74', '0.78', '0.83',
       '0.85', '0.87', '0.88', '0.91', '0.97', '0.99', '1.06', '1.07',
       '1.09', '1.18', '1.2', '1.25', '1.26', '1.28', '1.3', '1.43',
       '1.71', '1.85', '3.12', '3.23', '3.36'], dtype=object)

#### Fix Data Inconsistancy in *events* Column

In [24]:
# types of events
print("\nThe types of weather events in the 'events' column are:")
evnts = [str(x) for x in df_weather.events.unique()]
print("".join([str(i+1) + ". " + evnts[i] + "\n" for i in range(len(evnts))]))


The types of weather events in the 'events' column are:
1. nan
2. Fog
3. Rain
4. Fog-Rain
5. rain
6. Rain-Thunderstorm



The results above show that "Rain" value is entered in two different ways. We fix this by chaing "rain" values into "Rain".

In [26]:
# Change "rain" to "Rain" in the 'events' column .csv file
df_weather.loc[:, "events"] = df_weather.events.apply(lambda x: "Rain" if x == "rain" else x)

# types of events
print("\nThe types of weather events in the 'events' column are:")
evnts = [str(x) for x in df_weather.events.unique()]
print("".join([str(i+1) + ". " + evnts[i] + "\n" for i in range(len(evnts))]))


The types of weather events in the 'events' column are:
1. nan
2. Fog
3. Rain
4. Fog-Rain
5. Rain-Thunderstorm



## Store the Fixed Data

In [27]:
# write the fixed weather data in to weather_fixed.csv file
df.to_csv("../data/weather_fixed_2.csv", index=False)

## 2. Clean the *status* Data

### Display the original *status* Data

In [43]:
# read the first 5 rows of status data only for displying purpose.
file_path = "../data/status.csv"
df_status = pd.read_csv(file_path, nrows=100)

# get the total number of data points from sqlite db file
# make db connection
conn = sqlite3.connect("../data/database.sqlite")
cur = conn.cursor()

# get the total number of data points in status table
command = "SELECT Count(station_id) FROM {tb}".format(tb="status")
cur.execute(command)
npnts_1min = cur.fetchone()[0]

# close db connection
conn.close()

# display the data
display(df_status.head(10))

,station_id,bikes_available,docks_available,time
0,2,2,25,2013/08/29 12:06:01
1,2,2,25,2013/08/29 12:07:01
2,2,2,25,2013/08/29 12:08:01
3,2,2,25,2013/08/29 12:09:01
4,2,2,25,2013/08/29 12:10:01
5,2,2,25,2013/08/29 12:11:01
6,2,2,25,2013/08/29 12:12:01
7,2,2,25,2013/08/29 12:13:01
8,2,2,25,2013/08/29 12:15:01
9,2,2,25,2013/08/29 12:16:02


In [42]:
# Show the size of status.csv file
print "The file size of status.csv is " + str(os.path.getsize("../data/status.csv") >> 20) + " MB."
print("\nThe number of data points in this table is " + str(round(npnts_1min/1.e6, 2)) + " millions.")

The file size of status.csv is 1897 MB.

The number of data points in this table is 71.98 millions.


This file is quite large to work with in a normal PC.

### Downsample the *status* Data

In [ ]:
# run the folowing if downsampled data is needed. It only needs to be run once.
sample_data = False   # set this to True is sampled data is needed
db_to_csv = False     # set this to True if .csv format is desired

# set parameter values
time_res = 15
dbname = "../data/sampled_data.sqlite"
tbname = "time_res_" + str(time_res) + "min"

# downsample the data and then store the output in an sqlite3 db
if sample_data:
    output = sampler(time_res=time_res, n_jobs=None, save_to_db=True,
                     db_name=dbname, table_name=tbname, verbose=False)
    
# move data from sqlite3 db into a csv file, if needed
if db_to_csv:
    outfile = "../data/status_" + tbname + ".csv"
    print "Moving data from " + tbname + " table to a .csv file."
    sqlite_to_csv(dbname, tbname, outfile)
    print "status_time_res_" + str(time_res) + ".csv has been created."

### Display the Downsampled *status* Data

In [51]:
time_res = "15"

# make db connection
conn = sqlite3.connect("../data/sampled_data.sqlite")
cur = conn.cursor()

# read the data
df_status_res15 = pd.read_csv("../data/status_time_res_" + time_res + "min.csv", 
                              nrows=100, header=None, names=df_status.columns)

print("\nstatus data sampled at every " + time_res + " minutes:")
    
# display the data
display(df_status_res15.head(5))
    
# get the total number of data points in status table
tb = "time_res_" + time_res + "min"
command = "SELECT Count(station_id) FROM {tb}".format(tb=tb)
cur.execute(command)
npnts = cur.fetchone()[0]
ratio = round(100. * npnts / npnts_1min, 2)
print("\nThe number of data points in this table is " + str(round(npnts/1.e6, 2)) + " millions" +\
        ", which is " + str(ratio) + "% of the original data.\n")
    
# close db connection
conn.close()


status data sampled at every 15 minutes:


,station_id,bikes_available,docks_available,time
0,station_id,bikes_available,docks_available,time
1,2,2,25,2013-08-29 12:15:00
2,2,2,25,2013-08-29 12:30:00
3,2,2,25,2013-08-29 12:45:00
4,2,2,25,2013-08-29 13:00:00



The number of data points in this table is 4.8 millions, which is 6.67% of the original data.

